# Calculation of lambda inv. from monodromies of fibered knots


### Modules

In [ ]:
#import numpy as np
import pandas as pd
#import ast

#from multiprocessing import Pool, cpu_count
#from tqdm.notebook import tqdm

from snappy import *
#from GridPythonModule import *
#from GPM_wrapper.Grid import *

### automatic creation of non-alternating hyperbolic fibered knot data

In [ ]:
# prompt: create a data set of non-alternating knots up to 10 crossings as a list of dictionaries with keys 'name', 'monodromy' and 'genus_3D'.

def create_knot_data(name):
    """
    Creates a dictionary containing information about a knot.

    Args:
        name: The name of the knot.

    Returns:
        A dictionary with keys 'name', 'monodromy', and 'genus_3D'.
    """
    try:
        M = Manifold(name)
        monodromy = M.monodromy()
        genus_3D = M.genus()
        return {'name': name, 'monodromy': monodromy, 'genus_3D': genus_3D}
    except Exception as e:
        print(f"Error processing knot {name}: {e}")
        return None

def create_knot_dataset(max_crossings=10):
    """
    Creates a dataset of non-alternating knots up to a given number of crossings.

    Args:
        max_crossings: The maximum number of crossings for the knots.

    Returns:
        A list of dictionaries containing knot data.
    """
    knot_data = []
    for i in range(2, max_crossings + 1):
        for knot in Knot.names(i):
            if not Knot(knot).is_alternating():
                data = create_knot_data(knot)
                if data:
                    knot_data.append(data)
    return knot_data

# Create the dataset
knot_dataset = create_knot_dataset()


### a trial for using SnapPy and Twister

In [ ]:
#--- some hyperbolic fibered knots with genus up to 2
knot_data = [
 #-- alternating
 {'name': '4_1', 'md':'aB', 'alt': True},
 {'name': '6_2', 'md':'abcD', 'alt': True},
 {'name': '6_3', 'md':'abCD', 'alt': True},
 {'name': '7_7', 'md':'abCF', 'alt': True},
 {'name': '8_12', 'md':'aBcD', 'alt': True},
 #-- non-alternating
 {'name': '8_20', 'md':'abCDf', 'alt': False},
 {'name': '8_21', 'md':'abcDF', 'alt': False},
 {'name': '9_42', 'md':'abcdF', 'alt': False},
 {'name': '9_44', 'md':'abCDF', 'alt': False},
 ]

In [ ]:
for k in knot_data:
    print(f"{k=}")

    #--- create the exterior by name
    M1 = Manifold(k['name'])
    print(f"{M1.volume()=}") #, {M1.DT_code()=}")

    #--- create the exterior by monodromy
    S = twister.Surface('S_2_1')
    M2 = S.bundle(k['md'], optimize=True)
    try:
        L = M2.exterior_to_link()
        pd = L.PD_code()
        print(f"{M2.volume()=}, {pd=}")
        hfk_dic = hfk.pd_to_hfk(pd)
        print(f"{hfk_dic['ranks']=}")
    except:
        print("Error: exterior_to_link failed.")
        try:
            print(f"{M2.is_isometric_to(M1)=}")
        except:
            print('Can not determine')
    print(f"---")

In [ ]:
%%timeit
S = twister.Surface('S_2_1')
M3 = S.bundle(monodromy='a*b*C*D*F') #, optimize=True) #, return_type='Triangulation')
L = M3.exterior_to_link(verbose=False) #, simplify_link=False)
print(L.DT_code())
#print(f"{M3.volume()=}, {M3.DT_code()=}")

5.65 s ± 3.47

## Get HFK from monodromies

In [ ]:
S = twister.Surface('S_2_1')

mds = ['abeedcfEabC',
       # 'BECeADbDFEc',
       # 'cEfbabCDebD',
       # 'DebEFAceBDCe',
       # 'cAeABCFAFBDa',
       # 'DebEFAceBDCe',
       # 'BDabAAACebec',
       # 'afCeDafDbfbcABd',
       # 'FecfebfBEdBAcdfEA',
       # 'ceBEcaBCbEBBEcEFaa'
      ]

result = []
for md in mds:
    data = {'monodromy': md}
    M3 = S.bundle(monodromy=md)
    data['vol'] = M3.volume()

    L = M3.exterior_to_link() #verbose=True, simplify_link=False)
    #L.view()
    data['DT'] = L.DT_code()[0]
    data['PD'] = L.PD_code()

    hfk_dic = L.knot_floer_homology(); #print(f"{hfk_dic=}")
    data['lmd'] = list(hfk_dic['ranks'].keys())[-1][1]
    data['ranks'] = hfk_dic['ranks']
    result.append(data)

DF = pd.DataFrame(result); display(DF)
filename = f"./data/hyp_fib_g2_knots_with_neg_lmd.csv"
DF.to_csv(filename, mode = 'a', index=False, header = (not os.path.isfile(filename)))

## Scratch

In [ ]:
# @title An instance of a fibered knot with genus 2 that has negative lambda.
S = twister.Surface('S_2_1')

md = 'abeedcfEabC' #'DebEFAceBDCe' #'BECeADbDFEc' #'DebEFAceBDCe' #'FecfebfBEdBAcdfEA' #'afCeDafDbfbcABd' #'cAeABCFAFBDa' #'ceBEcaBCbEBBEcEFaa' #
print(f"{md}") 

M3 = S.bundle(monodromy=md)
print(M3.volume())

L = M3.exterior_to_link()
#L.view()
print(len(L.DT_code()[0]))
#pd = L.PD_code(); print(pd)

hfk = L.knot_floer_homology()
lmd = list(hfk['ranks'].keys())[-1][1]; print(f"{lmd=}")